# Execution Plan

In this notebook we try to understand execution plans. We will use the weather example and analyse all the steps.

# 1. Load Data

In [2]:
storageLocation = "s3://dimajix-training/data/weather"

## 1.1 Load Measurements

In [4]:
from pyspark.sql.functions import *
from functools import reduce

# Read in all years, store them in an Python array
raw_weather_per_year = [spark.read.text(storageLocation + "/" + str(i)).withColumn("year", lit(i)) for i in range(2003,2015)]

# Union all years together
raw_weather = reduce(lambda l,r: l.union(r), raw_weather_per_year)                        

# Display first 10 records
raw_weather.limit(10).toPandas()

,value,year
0,0494703160256242003010100003+55200-162717SY-MT...,2003
1,0228703160256242003010100174+55200-162730FM-16...,2003
2,044070316025624200301010053C+55200-162717FM-15...,2003
3,0071703160256242003010101009+55200-162717NSRDB...,2003
4,042770316025624200301010153C+55200-162717FM-15...,2003
5,0071703160256242003010102009+55200-162717NSRDB...,2003
6,046870316025624200301010253C+55200-162717FM-15...,2003
7,0071703160256242003010103009+55200-162717NSRDB...,2003
8,041570316025624200301010353C+55200-162717FM-15...,2003
9,0054703160256242003010104009+55200-162717NSRDB...,2003


In [5]:
weather = raw_weather.select(
    col("year"),
    substring(col("value"),5,6).alias("usaf"),
    substring(col("value"),11,5).alias("wban"),
    substring(col("value"),16,8).alias("date"),
    substring(col("value"),24,4).alias("time"),
    substring(col("value"),42,5).alias("report_type"),
    substring(col("value"),61,3).alias("wind_direction"),
    substring(col("value"),64,1).alias("wind_direction_qual"),
    substring(col("value"),65,1).alias("wind_observation"),
    (substring(col("value"),66,4).cast("float") / lit(10.0)).alias("wind_speed"),
    substring(col("value"),70,1).alias("wind_speed_qual"),
    (substring(col("value"),88,5).cast("float") / lit(10.0)).alias("air_temperature"),
    substring(col("value"),93,1).alias("air_temperature_qual")
)
    
weather.limit(10).toPandas()

,year,usaf,wban,date,time,report_type,wind_direction,wind_direction_qual,wind_observation,wind_speed,wind_speed_qual,air_temperature,air_temperature_qual
0,2003,703160,25624,20030101,0000,SY-MT,010,5,N,5.2,5,-0.6,5
1,2003,703160,25624,20030101,0017,FM-16,020,1,N,4.6,1,-2.0,1
2,2003,703160,25624,20030101,0053,FM-15,010,5,N,5.2,5,-2.8,5
3,2003,703160,25624,20030101,0100,NSRDB,999,9,9,999.9,9,999.9,9
4,2003,703160,25624,20030101,0153,FM-15,010,5,N,6.2,5,-2.2,5
5,2003,703160,25624,20030101,0200,NSRDB,999,9,9,999.9,9,999.9,9
6,2003,703160,25624,20030101,0253,FM-15,010,5,N,7.2,5,-3.3,5
7,2003,703160,25624,20030101,0300,NSRDB,999,9,9,999.9,9,999.9,9
8,2003,703160,25624,20030101,0353,FM-15,020,5,N,6.2,5,-1.1,5
9,2003,703160,25624,20030101,0400,NSRDB,999,9,9,999.9,9,999.9,9


## 1.2 Load Station Metadata

In [6]:
stations = spark.read \
    .option("header", True) \
    .csv(storageLocation + "/isd-history")

# Display first 10 records    
stations.limit(10).toPandas()

,USAF,WBAN,STATION NAME,CTRY,STATE,ICAO,LAT,LON,ELEV(M),BEGIN,END
0,007005,99999,CWOS 07005,None,None,None,None,None,None,20120127,20120127
1,007011,99999,CWOS 07011,None,None,None,None,None,None,20111025,20121129
2,007018,99999,WXPOD 7018,None,None,None,+00.000,+000.000,+7018.0,20110309,20130730
3,007025,99999,CWOS 07025,None,None,None,None,None,None,20120127,20120127
4,007026,99999,WXPOD 7026,AF,None,None,+00.000,+000.000,+7026.0,20120713,20141120
5,007034,99999,CWOS 07034,None,None,None,None,None,None,20121024,20121106
6,007037,99999,CWOS 07037,None,None,None,None,None,None,20111202,20121125
7,007044,99999,CWOS 07044,None,None,None,None,None,None,20120127,20120127
8,007047,99999,CWOS 07047,None,None,None,None,None,None,20120613,20120717
9,007052,99999,CWOS 07052,None,None,None,None,None,None,20121129,20121130


## 1.3 Perform Analysis

In [7]:
df = weather.join(stations, (weather.usaf == stations.USAF) & (weather.wban == stations.WBAN))
result = df.groupBy(df.CTRY, df.year).agg(
        min(when(df.air_temperature_qual == lit(1), df.air_temperature)).alias('min_temp'),
        max(when(df.air_temperature_qual == lit(1), df.air_temperature)).alias('max_temp'),
        min(when(df.wind_speed_qual == lit(1), df.wind_speed)).alias('min_wind'),
        max(when(df.wind_speed_qual == lit(1), df.wind_speed)).alias('max_wind')
    )

pdf = result.toPandas()    
pdf

,CTRY,year,min_temp,max_temp,min_wind,max_wind
0,CA,2006,-43.0,35.9,0.0,31.4
1,SC,2006,20.0,32.0,0.0,20.6
2,IC,2013,-11.0,19.0,0.0,25.0
3,AM,2011,-16.0,42.0,0.0,14.0
4,UK,2014,-6.0,30.4,0.0,20.6
5,SF,2012,2.2,36.0,0.0,10.3
6,GM,2005,-14.0,31.0,0.0,14.4
7,NO,2007,-35.0,29.0,0.0,26.0
8,PO,2010,-1.6,38.0,0.0,21.6
9,FR,2010,-13.3,36.1,0.0,17.5


# 2 Investigate Execution Plans

## 2.1 Reading Data

In [12]:
raw_weather_2003 = spark.read.text(storageLocation + "/2003")
raw_weather_2003.explain()

== Physical Plan ==
*(1) FileScan text [value#321] Batched: false, Format: Text, Location: InMemoryFileIndex[s3://dimajix-training/data/weather/2003], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<value:string>


In [24]:
raw_weather_2003.explain(True)

== Parsed Logical Plan ==
Project [value#323, 2003 AS year#325]
+- AnalysisBarrier
      +- Relation[value#323] text

== Analyzed Logical Plan ==
value: string, year: int
Project [value#323, 2003 AS year#325]
+- Relation[value#323] text

== Optimized Logical Plan ==
Project [value#323, 2003 AS year#325]
+- Relation[value#323] text

== Physical Plan ==
*(1) Project [value#323, 2003 AS year#325]
+- *(1) FileScan text [value#323] Batched: false, Format: Text, Location: InMemoryFileIndex[s3://dimajix-training/data/weather/2003], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<value:string>


## 2.2 Adding Columns

In [25]:
raw_weather_2003 = spark.read.text(storageLocation + "/2003").withColumn("year", lit(2003))
raw_weather_2003.explain(True)

== Parsed Logical Plan ==
Project [value#664, 2003 AS year#666]
+- AnalysisBarrier
      +- Relation[value#664] text

== Analyzed Logical Plan ==
value: string, year: int
Project [value#664, 2003 AS year#666]
+- Relation[value#664] text

== Optimized Logical Plan ==
Project [value#664, 2003 AS year#666]
+- Relation[value#664] text

== Physical Plan ==
*(1) Project [value#664, 2003 AS year#666]
+- *(1) FileScan text [value#664] Batched: false, Format: Text, Location: InMemoryFileIndex[s3://dimajix-training/data/weather/2003], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<value:string>


## 2.3 SELECT Operation

In [34]:
weather_2003 = raw_weather_2003.select(
    col("year"),
    substring(col("value"),5,6).alias("usaf"),
    substring(col("value"),11,5).alias("wban")
)
weather_2003.explain(True)

== Parsed Logical Plan ==
'Project [unresolvedalias('year, None), substring('value, 5, 6) AS usaf#949, substring('value, 11, 5) AS wban#950]
+- AnalysisBarrier
      +- Project [value#664, 2003 AS year#666]
         +- Relation[value#664] text

== Analyzed Logical Plan ==
year: int, usaf: string, wban: string
Project [year#666, substring(value#664, 5, 6) AS usaf#949, substring(value#664, 11, 5) AS wban#950]
+- Project [value#664, 2003 AS year#666]
   +- Relation[value#664] text

== Optimized Logical Plan ==
Project [2003 AS year#666, substring(value#664, 5, 6) AS usaf#949, substring(value#664, 11, 5) AS wban#950]
+- Relation[value#664] text

== Physical Plan ==
*(1) Project [2003 AS year#666, substring(value#664, 5, 6) AS usaf#949, substring(value#664, 11, 5) AS wban#950]
+- *(1) FileScan text [value#664] Batched: false, Format: Text, Location: InMemoryFileIndex[s3://dimajix-training/data/weather/2003], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<value:string>


Note how the optimizer merged both select operations into a single one

## 2.4 UNION Operation

In [35]:
raw_weather.explain()

== Physical Plan ==
Union
:- *(1) Project [value#2, 2003 AS year#4]
:  +- *(1) FileScan text [value#2] Batched: false, Format: Text, Location: InMemoryFileIndex[s3://dimajix-training/data/weather/2003], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<value:string>
:- *(2) Project [value#7, 2004 AS year#9]
:  +- *(2) FileScan text [value#7] Batched: false, Format: Text, Location: InMemoryFileIndex[s3://dimajix-training/data/weather/2004], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<value:string>
:- *(3) Project [value#12, 2005 AS year#14]
:  +- *(3) FileScan text [value#12] Batched: false, Format: Text, Location: InMemoryFileIndex[s3://dimajix-training/data/weather/2005], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<value:string>
:- *(4) Project [value#17, 2006 AS year#19]
:  +- *(4) FileScan text [value#17] Batched: false, Format: Text, Location: InMemoryFileIndex[s3://dimajix-training/data/weather/2006], PartitionFilters: [], PushedFilters: [

## 2.5 JOIN Operation

In [36]:
df = weather_2003.join(stations, (weather_2003.usaf == stations.USAF) & (weather_2003.wban == stations.WBAN))
df.explain(True)

== Parsed Logical Plan ==
Join Inner, ((usaf#949 = USAF#145) && (wban#950 = WBAN#146))
:- Project [year#666, substring(value#664, 5, 6) AS usaf#949, substring(value#664, 11, 5) AS wban#950]
:  +- Project [value#664, 2003 AS year#666]
:     +- Relation[value#664] text
+- Relation[USAF#145,WBAN#146,STATION NAME#147,CTRY#148,STATE#149,ICAO#150,LAT#151,LON#152,ELEV(M)#153,BEGIN#154,END#155] csv

== Analyzed Logical Plan ==
year: int, usaf: string, wban: string, USAF: string, WBAN: string, STATION NAME: string, CTRY: string, STATE: string, ICAO: string, LAT: string, LON: string, ELEV(M): string, BEGIN: string, END: string
Join Inner, ((usaf#949 = USAF#145) && (wban#950 = WBAN#146))
:- Project [year#666, substring(value#664, 5, 6) AS usaf#949, substring(value#664, 11, 5) AS wban#950]
:  +- Project [value#664, 2003 AS year#666]
:     +- Relation[value#664] text
+- Relation[USAF#145,WBAN#146,STATION NAME#147,CTRY#148,STATE#149,ICAO#150,LAT#151,LON#152,ELEV(M)#153,BEGIN#154,END#155] csv

== Opt

Things to note:
* Spark filters columns, since a JOIN require non-null values
* Filtering is actually pushed down before the projection. This reduces amount of data as soon as possible
* JOIN operation is performed using a broadcast JOIN (more on that later)

## 2.6 Aggregation

In [39]:
weather_2003 = raw_weather_2003.select(
    col("year"),
    substring(col("value"),5,6).alias("usaf"),
    substring(col("value"),11,5).alias("wban"),
    substring(col("value"),16,8).alias("date"),
    substring(col("value"),24,4).alias("time"),
    substring(col("value"),42,5).alias("report_type"),
    substring(col("value"),61,3).alias("wind_direction"),
    substring(col("value"),64,1).alias("wind_direction_qual"),
    substring(col("value"),65,1).alias("wind_observation"),
    (substring(col("value"),66,4).cast("float") / lit(10.0)).alias("wind_speed"),
    substring(col("value"),70,1).alias("wind_speed_qual"),
    (substring(col("value"),88,5).cast("float") / lit(10.0)).alias("air_temperature"),
    substring(col("value"),93,1).alias("air_temperature_qual")
)
weather_2003.explain(False)

== Physical Plan ==
*(1) Project [2003 AS year#666, substring(value#664, 5, 6) AS usaf#1093, substring(value#664, 11, 5) AS wban#1094, substring(value#664, 16, 8) AS date#1095, substring(value#664, 24, 4) AS time#1096, substring(value#664, 42, 5) AS report_type#1097, substring(value#664, 61, 3) AS wind_direction#1098, substring(value#664, 64, 1) AS wind_direction_qual#1099, substring(value#664, 65, 1) AS wind_observation#1100, (cast(cast(substring(value#664, 66, 4) as float) as double) / 10.0) AS wind_speed#1101, substring(value#664, 70, 1) AS wind_speed_qual#1102, (cast(cast(substring(value#664, 88, 5) as float) as double) / 10.0) AS air_temperature#1103, substring(value#664, 93, 1) AS air_temperature_qual#1104]
+- *(1) FileScan text [value#664] Batched: false, Format: Text, Location: InMemoryFileIndex[s3://dimajix-training/data/weather/2003], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<value:string>


In [40]:
df = weather_2003.join(stations, (weather_2003.usaf == stations.USAF) & (weather_2003.wban == stations.WBAN))
df.explain(False)

== Physical Plan ==
*(2) BroadcastHashJoin [usaf#1093, wban#1094], [USAF#145, WBAN#146], Inner, BuildRight
:- *(2) Project [2003 AS year#666, substring(value#664, 5, 6) AS usaf#1093, substring(value#664, 11, 5) AS wban#1094, substring(value#664, 16, 8) AS date#1095, substring(value#664, 24, 4) AS time#1096, substring(value#664, 42, 5) AS report_type#1097, substring(value#664, 61, 3) AS wind_direction#1098, substring(value#664, 64, 1) AS wind_direction_qual#1099, substring(value#664, 65, 1) AS wind_observation#1100, (cast(cast(substring(value#664, 66, 4) as float) as double) / 10.0) AS wind_speed#1101, substring(value#664, 70, 1) AS wind_speed_qual#1102, (cast(cast(substring(value#664, 88, 5) as float) as double) / 10.0) AS air_temperature#1103, substring(value#664, 93, 1) AS air_temperature_qual#1104]
:  +- *(2) Filter (isnotnull(substring(value#664, 11, 5)) && isnotnull(substring(value#664, 5, 6)))
:     +- *(2) FileScan text [value#664] Batched: false, Format: Text, Location: InMemor

In [41]:
result = df.groupBy(df.CTRY, df.year).agg(
        min(when(df.air_temperature_qual == lit(1), df.air_temperature)).alias('min_temp'),
        max(when(df.air_temperature_qual == lit(1), df.air_temperature)).alias('max_temp'),
        min(when(df.wind_speed_qual == lit(1), df.wind_speed)).alias('min_wind'),
        max(when(df.wind_speed_qual == lit(1), df.wind_speed)).alias('max_wind')
    )
result.explain(True)

== Parsed Logical Plan ==
'Aggregate [CTRY#148, year#666], [CTRY#148, year#666, min(CASE WHEN (air_temperature_qual#1104 = 1) THEN air_temperature#1103 END) AS min_temp#1215, max(CASE WHEN (air_temperature_qual#1104 = 1) THEN air_temperature#1103 END) AS max_temp#1217, min(CASE WHEN (wind_speed_qual#1102 = 1) THEN wind_speed#1101 END) AS min_wind#1219, max(CASE WHEN (wind_speed_qual#1102 = 1) THEN wind_speed#1101 END) AS max_wind#1221]
+- AnalysisBarrier
      +- Join Inner, ((usaf#1093 = USAF#145) && (wban#1094 = WBAN#146))
         :- Project [year#666, substring(value#664, 5, 6) AS usaf#1093, substring(value#664, 11, 5) AS wban#1094, substring(value#664, 16, 8) AS date#1095, substring(value#664, 24, 4) AS time#1096, substring(value#664, 42, 5) AS report_type#1097, substring(value#664, 61, 3) AS wind_direction#1098, substring(value#664, 64, 1) AS wind_direction_qual#1099, substring(value#664, 65, 1) AS wind_observation#1100, (cast(cast(substring(value#664, 66, 4) as float) as double)

Things to note
* Projections only contains the columns required, not all available columns of df
* Aggregation is performed in three steps: 
  * Partial aggregation
  * Shuffle
  * Final aggregation of partial results